In [1]:
# Preface:
# Goal Disassemble: 
#   1. final goal: net load = total load - solar energy
#   2. current data covers 4 parts:
#      1) au_adelaide_act: weather related features data in adelaide
#      2) au_sa_iso_satellite_pv_load_act: solar behind-the-meter generation      
#      3) au_sa_load_act: net load
#      4) au_sa_total_load_act + au_sa_total_load_act_202211-202303: total load
            
# Initial guess:
# Based on current data, it would be difficult to predict net load or total load
# directly based on data 3 or data 4, because they only have time related 
# dimensions and lack of useful variables as input. Therefore, firstly I will
# start with 'solar energy' part in the final goal, which can be predicted based
# on the rich series of weather features in data 1 as training input and the solor 
# energy generation in data 2 as training output.


# 1. load the 2 csv dataset:
# store file path
import pandas as pd
weather_path = "../data/au_adelaide_act.csv"
solar_path = "../data/au_sa_iso_satellite_pv_load_act.csv"

# load and store dataset
weather = pd.read_csv(weather_path)
solar = pd.read_csv(solar_path)


# 2. Missing values (-99):
# replace missing values '-99' with NaN
import numpy as np
weather.replace(-99, np.nan, inplace = True)
solar.replace(-99, np.nan, inplace = True)

# for 'weather'(au_adelaide_act) dataset, the NaN counts of each column is:
nan_counts_weather = weather.isna().sum()

print("Number of NaN values in each column in weather data:")
print("Row count:", len(weather))
print(nan_counts_weather)




Number of NaN values in each column in weather data:
Row count: 62960
date          0
time          0
tempc         5
cloud8       18
windk         5
wdir          5
humid         5
rainmm       33
radkjm2    6398
dtype: int64


In [2]:
# 2.1 Weather:
# From the result above, we notice that around 10% (6398/62960) of the radiation('radkjm2') 
# records are missing, while radiation is naturally one of the most important factors that
# affect the solor panel energy generation. So I make a summary specifically for radiation
# nan by year and see more details:
from datetime import datetime

# copy weather as 'weather2' for radiation nan analysis
weather2 = weather

# extract [year] from [date] column
weather2['date'] = weather2['date'].astype(str)
weather2['year'] = weather2['date'].str.slice(start=0, stop=4)


# count na & non-na values in radiation
weather2['rad_is_nan'] = weather2['radkjm2'].isna()

non_nan_count = weather.groupby('year').agg(non_nan = ('radkjm2', 'count'))
radiation_records_by_year = weather.groupby('year').agg(total_records = ('year', 'count'))

# test = weather.groupby('year').count().reset_index()
radia_na_summary = pd.merge(radiation_records_by_year, non_nan_count, on = 'year')

print("Summary for missing values in radiation: \n", radia_na_summary)

Summary for missing values in radiation: 
       total_records  non_nan
year                        
2016           8784     2391
2017           8760     8760
2018           8760     8755
2019           8760     8760
2020           8784     8784
2021           8760     8760
2022           8760     8760
2023           1592     1592


In [3]:
# Summary:
# From the result above, the radiaion has many more nan values in 2016 tham the other years
# and nearly occupies 2/3 of the whole year data, I would prefer to drop these records directly
# since they are missing because of external factors, but to leave the na in other years there 
# to ensure the the generalization to test data of the final model will not be affected.

# drop the na related staff from the original data and store as final weather data
weather = weather.dropna(subset = ['radkjm2']).drop('rad_is_nan', axis = 1)
weather

# (If you guys have any better suggestions for data cleaning, feel free and please tell me.)

,date,time,tempc,cloud8,windk,wdir,humid,rainmm,radkjm2,year
6393,20160923,900,16.0,3.0,15.9,20.0,48.0,0.00,201.0,2016
6394,20160923,1000,16.6,3.0,16.9,20.0,47.0,0.00,264.0,2016
6395,20160923,1100,18.6,3.0,16.0,20.0,41.0,0.00,310.0,2016
6396,20160923,1200,18.8,2.0,15.9,20.0,44.0,0.00,334.0,2016
6397,20160923,1300,19.3,2.0,14.0,30.0,44.0,0.00,324.0,2016
...,...,...,...,...,...,...,...,...,...,...
62955,20230308,300,16.1,8.0,17.0,260.0,87.0,0.10,0.0,2023
62956,20230308,400,17.0,8.0,15.0,260.0,79.0,0.71,0.0,2023
62957,20230308,500,16.4,8.0,15.0,260.0,87.0,0.54,0.0,2023
62958,20230308,600,15.9,7.0,13.0,260.0,89.0,0.49,20.0,2023


In [4]:
# 2.2 Solar:
# for 'solar'(au_sa_iso_satellite_pv_load_act) dataset, 
# the NaN counts of each column is:
nan_counts_solar = solar.isna().sum()
print("\nNumber of NaN values in each column in solar data:")
print("Row count:", len(solar))
print(nan_counts_solar)


Number of NaN values in each column in solar data:
Row count: 87741
date           0
time           0
load_act    1098
dtype: int64


In [5]:
# Summary for Solar:
# Since the number of nan values in [load_act] is relatively low, wtih only 1098 nan in the whole 
# 87741 records, I would prefer to keep them there.
# (The earliest data for solar generation dates back to 2018 while the weather data starts from 2016.
# Currently we may just leave this condition there. In the future I will go to the data soure 
# AEMO website to see wether we need to fill in the 2016-2018 data for solar generations.)

In [6]:
# 3. Join weather & solar generation
# According the report definitions from AEMO website, [time] in weather reprents the weather 
# conditions at each time point, while the [time] in solar generation recorded by sateelite (30 min
# delay) means the solar generation between each time point. Therefore, to unify the [time], we may
# need to add a new column represents each time point instead of original time interval in the solar
# generation dataset.

# Do the quick cleanning for the solar generation hour and minutes
def convert_hour(time_str):
    time_str = str(time_str)
    if time_str == '0':
        return '0'
    elif time_str == '30':
        return '0'
    elif len(time_str) == 3:
        return time_str[0]
    elif len(time_str) == 4:
        return time_str[:2]
    else:
        return time_str
def convert_minute(time_str):
    time_str = str(time_str)
    if time_str == '0':
        return '00'
    elif time_str == '30':
        return '30'
    elif len(time_str) == 3:
        return time_str[1:]
    elif len(time_str) == 4:
        return time_str[2:]
    else:
        return time_str
solar['formatted_hour'] = solar['time'].apply(convert_hour)
solar['formatted_minute'] = solar['time'].apply(convert_minute)
solar['solar_date'] = solar['date'].astype(str)
solar['formatted_month'] = solar['solar_date'].str.slice(start=4, stop=6)
solar['formatted_day'] = solar['solar_date'].str.slice(start=6, stop=8)
solar.drop(['date','time'], axis=1, inplace=True)


weather['formatted_hour'] = weather['time'].apply(convert_hour)
weather['weather_date'] = weather['date'].astype(str)
weather.drop(['date','time'], axis=1, inplace=True)

In [7]:
# print(solar['formatted_hour'].unique())

In [8]:
# print(weather['formatted_hour'].unique())

In [9]:
# Now join the solar generation and weather data
mix_data = pd.merge(weather,solar,how='inner',left_on=['weather_date','formatted_hour'],right_on=['solar_date','formatted_hour'])
mix_data = mix_data[['weather_date','year','formatted_month','formatted_day','formatted_hour',
                      'formatted_minute','tempc','cloud8','windk','wdir','humid','rainmm','radkjm2']]
# Now we have done the basic cleaning and merging, the usable Xs and Y are ready, and we can move 

In [10]:
# to build the predition model of the solar generation. 
mix_data.head(10)

,weather_date,year,formatted_month,formatted_day,formatted_hour,formatted_minute,tempc,cloud8,windk,wdir,humid,rainmm,radkjm2
0,20180306,2018,03,06,9,30,20.0,4.0,13.0,130.0,49.0,0.0,1490.0
1,20180306,2018,03,06,10,00,21.5,1.0,16.0,140.0,40.0,0.0,2340.0
2,20180306,2018,03,06,10,30,21.5,1.0,16.0,140.0,40.0,0.0,2340.0
3,20180306,2018,03,06,11,00,23.0,2.0,15.0,150.0,34.0,0.0,2800.0
4,20180306,2018,03,06,11,30,23.0,2.0,15.0,150.0,34.0,0.0,2800.0
5,20180306,2018,03,06,12,00,24.1,2.0,11.0,140.0,30.0,0.0,3090.0
6,20180306,2018,03,06,12,30,24.1,2.0,11.0,140.0,30.0,0.0,3090.0
7,20180306,2018,03,06,13,00,25.3,0.0,17.0,150.0,26.0,0.0,3110.0
8,20180306,2018,03,06,13,30,25.3,0.0,17.0,150.0,26.0,0.0,3110.0
9,20180306,2018,03,06,14,00,25.3,1.0,14.0,140.0,29.0,0.0,2850.0


In [11]:
mix_data.to_csv('../data/weather_solar_merge.csv', index=False)

In [12]:
weather['formatted_minute'] = '00'
#pd.merge(weather, , how='inner', left_on)
weather30 = weather.drop(weather.columns[:7], axis=1)
weather30['formatted_minute'] = '30'
weather30[weather.columns[:7]] = None
weather_interpolate = pd.concat([weather,  weather30], ignore_index=True).sort_values(
    by=['weather_date', 'formatted_hour', 'formatted_minute'])
weather_interpolate

,tempc,cloud8,windk,wdir,humid,rainmm,radkjm2,year,formatted_hour,weather_date,formatted_minute
1,16.6,3.0,16.9,20.0,47.0,0.00,264.0,2016,10,20160923,00
56563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,10,20160923,30
2,18.6,3.0,16.0,20.0,41.0,0.00,310.0,2016,11,20160923,00
56564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,11,20160923,30
3,18.8,2.0,15.9,20.0,44.0,0.00,334.0,2016,12,20160923,00
...,...,...,...,...,...,...,...,...,...,...,...
113121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,5,20230308,30
56560,15.9,7.0,13.0,260.0,89.0,0.49,20.0,2023,6,20230308,00
113122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,6,20230308,30
56561,16.0,8.0,17.0,230.0,88.0,0.32,150.0,2023,7,20230308,00


In [13]:
weather_interpolate.interpolate(inplace=True)
weather_interpolate

,tempc,cloud8,windk,wdir,humid,rainmm,radkjm2,year,formatted_hour,weather_date,formatted_minute
1,16.60,3.0,16.90,20.0,47.0,0.000,264.0,2016,10,20160923,00
56563,17.60,3.0,16.45,20.0,44.0,0.000,287.0,2016,10,20160923,30
2,18.60,3.0,16.00,20.0,41.0,0.000,310.0,2016,11,20160923,00
56564,18.70,2.5,15.95,20.0,42.5,0.000,322.0,2016,11,20160923,30
3,18.80,2.0,15.90,20.0,44.0,0.000,334.0,2016,12,20160923,00
...,...,...,...,...,...,...,...,...,...,...,...
113121,16.15,7.5,14.00,260.0,88.0,0.515,10.0,2023,5,20230308,30
56560,15.90,7.0,13.00,260.0,89.0,0.490,20.0,2023,6,20230308,00
113122,15.95,7.5,15.00,245.0,88.5,0.405,85.0,2023,6,20230308,30
56561,16.00,8.0,17.00,230.0,88.0,0.320,150.0,2023,7,20230308,00


In [14]:
mix_interpolate = pd.merge(weather,solar,how='inner',
                           left_on=['weather_date','formatted_hour','formatted_minute'],
                           right_on=['solar_date','formatted_hour','formatted_minute'], copy=False)
mix_interpolate = mix_interpolate[['weather_date','year','formatted_month','formatted_day','formatted_hour',
                      'formatted_minute','tempc','cloud8','windk','wdir','humid','rainmm','radkjm2']]
mix_interpolate

,weather_date,year,formatted_month,formatted_day,formatted_hour,formatted_minute,tempc,cloud8,windk,wdir,humid,rainmm,radkjm2
0,20180306,2018,03,06,10,00,21.5,1.0,16.0,140.0,40.0,0.00,2340.0
1,20180306,2018,03,06,11,00,23.0,2.0,15.0,150.0,34.0,0.00,2800.0
2,20180306,2018,03,06,12,00,24.1,2.0,11.0,140.0,30.0,0.00,3090.0
3,20180306,2018,03,06,13,00,25.3,0.0,17.0,150.0,26.0,0.00,3110.0
4,20180306,2018,03,06,14,00,25.3,1.0,14.0,140.0,29.0,0.00,2850.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43860,20230308,2023,03,08,3,00,16.1,8.0,17.0,260.0,87.0,0.10,0.0
43861,20230308,2023,03,08,4,00,17.0,8.0,15.0,260.0,79.0,0.71,0.0
43862,20230308,2023,03,08,5,00,16.4,8.0,15.0,260.0,87.0,0.54,0.0
43863,20230308,2023,03,08,6,00,15.9,7.0,13.0,260.0,89.0,0.49,20.0


In [15]:
mix_interpolate.to_csv('../data/weather_solar_interpolate.csv', index=False)

In [16]:
solar

,load_act,formatted_hour,formatted_minute,solar_date,formatted_month,formatted_day
0,318.991,9,30,20180306,03,06
1,375.231,10,00,20180306,03,06
2,430.909,10,30,20180306,03,06
3,485.129,11,00,20180306,03,06
4,523.989,11,30,20180306,03,06
...,...,...,...,...,...,...
87736,0.000,5,30,20230308,03,08
87737,0.000,6,00,20230308,03,08
87738,0.328,6,30,20230308,03,08
87739,31.739,7,00,20230308,03,08


In [17]:
weather_interpolate

,tempc,cloud8,windk,wdir,humid,rainmm,radkjm2,year,formatted_hour,weather_date,formatted_minute
1,16.60,3.0,16.90,20.0,47.0,0.000,264.0,2016,10,20160923,00
56563,17.60,3.0,16.45,20.0,44.0,0.000,287.0,2016,10,20160923,30
2,18.60,3.0,16.00,20.0,41.0,0.000,310.0,2016,11,20160923,00
56564,18.70,2.5,15.95,20.0,42.5,0.000,322.0,2016,11,20160923,30
3,18.80,2.0,15.90,20.0,44.0,0.000,334.0,2016,12,20160923,00
...,...,...,...,...,...,...,...,...,...,...,...
113121,16.15,7.5,14.00,260.0,88.0,0.515,10.0,2023,5,20230308,30
56560,15.90,7.0,13.00,260.0,89.0,0.490,20.0,2023,6,20230308,00
113122,15.95,7.5,15.00,245.0,88.5,0.405,85.0,2023,6,20230308,30
56561,16.00,8.0,17.00,230.0,88.0,0.320,150.0,2023,7,20230308,00
